In [ ]:
# started automation onf 3 Sep 2024 19:15
import configparser, logging, time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime, timedelta
from persist.database import SensorDataHandler
from visualize.plots import draw_plots
from sensors.camera import RpiCamController, DIMENSION_FULL_HD, DIMENSION_CAMERA_DEFAULT
from sensors.dht import DHT, DHTResult
_config = configparser.ConfigParser()
_config.read('hometemp.ini')
log = logging.getLogger('crunch_numbers')
auth = _config["db"]

In [ ]:
def get_sensor_data(used_pin):
    """
    Returns temperature and humidity data measures by DHT11 Sensor and the measurement timestamp.
    """
    max_tries = 20
    tries = 0
    DHT_SENSOR = DHT(used_pin, True)
    while True:
        if tries >= max_tries:
            log.error(f"Failed to retrieve data from DHT11 sensor: Maximum retries reached.")
            break
        else:
            time.sleep(2)
            result = DHT_SENSOR.read()
            if result.error_code == DHTResult.ERR_NOT_FOUND:
                tries += 1
                log.warning(f"({tries}/{max_tries}) Sensor could not be found. Using correct pin?")
                # maybe exit here but invastige when this error occurs (should never)
                continue
            elif not result.is_valid():
                tries += 1
                log.warning(f"({tries}/{max_tries}) Sensor data invalid")
                continue
            elif result.is_valid() and result.error_code == DHTResult.ERR_NO_ERROR:
                # postgres expects timestamp ins ISO 8601 format
                timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                return result.temperature, result.humidity, timestamp
    
    return None, None, None

#live_sensor_data = get_sensor_data(int(_config["hometemp"]["sensor_pin"]))
#live_sensor_data

In [ ]:
rpi_cam = RpiCamController()
#name = f'pictures/{datetime.now().strftime("%Y-%m-%d-%H:%M:%S")}'
name = "current"
rpi_cam.capture_image(file_path=name, rotation=90, dimension=DIMENSION_CAMERA_DEFAULT)

In [ ]:
handler = SensorDataHandler(auth['db_port'], auth['db_host'], auth['db_user'], auth['db_pw'], 'sensor_data')
handler.init_db_connection(check_table=False)
df = handler.read_data_into_dataframe()
df['timestamp'] = df['timestamp'].map(lambda x : datetime.strptime(str(x).replace("+00:00", "").strip(), '%Y-%m-%d %H:%M:%S'))
df = df.sort_values(by='timestamp')
df = df.reset_index()

df

In [ ]:
# last two hours
#df[["timestamp", "room_temp"]].tail(12)

In [ ]:
df[["humidity", "room_temp", "cpu_temp"]].describe()

In [ ]:
corr_matrix = df[["humidity", "room_temp", "cpu_temp"]].corr()
#corr_matrix = df[["humidity", "room_temp"]].corr()
corr_matrix

In [ ]:
draw_plots(df, with_save=False)

In [ ]:
plt.figure(figsize=(25, 10))
plt.subplot(2, 1, 1)

sns.lineplot(label="CPU", x="timestamp", y="cpu_temp", data=df)
plt.title("Temperature Over Time")
plt.xlabel("Time")
plt.ylabel("Temp (°C)")
plt.legend()
plt.xticks(rotation=45)

plt.subplot(2, 1, 2)
sns.lineplot(label="CPU", x="timestamp", y="cpu_temp",marker='o',markersize=6, data=df[df["timestamp"] >= datetime.now() - timedelta(hours=25)])
plt.title("Temperature Last 24h")
plt.xlabel("Time")
plt.ylabel("Temp (°C)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
